<a href="https://colab.research.google.com/github/Hernanros/SOTA/blob/master/notebooks/labels/WMD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd "/content/drive/My Drive/WORK/ML/YDATA/NLP"

In [4]:
!pip install wmd

     |████████████████████████████████| 112kB 5.2MB/s 
  Created wheel for wmd: filename=wmd-1.3.2-cp36-cp36m-linux_x86_64.whl size=629317 sha256=374f473f3ed80757c210b07b1312ce010b5d3823531812479b0f613952b4eb4d
  Stored in directory: /root/.cache/pip/wheels/a6/4d/7a/fc3fdbc12e33c0551301543e4441fb4783c809a30451387ffd
Successfully built wmd


In [5]:
!pip install wordfreq

     |████████████████████████████████| 32.8MB 112kB/s 
     |████████████████████████████████| 4.9MB 51.0MB/s 
     |████████████████████████████████| 276kB 43.9MB/s 
  Created wheel for wordfreq: filename=wordfreq-2.3.2-cp36-none-any.whl size=32817238 sha256=43f22d88e805aadb0f2e8feab0d6a4020d69c9dd93c7ce49d3ea4fbc61ff40d2
  Stored in directory: /root/.cache/pip/wheels/8d/ba/84/ba6be76208bd2c2124b6586f7967fb87e9f9fb4b4827e5e2c9
  Created wheel for langcodes: filename=langcodes-2.0.0-cp36-none-any.whl size=5044047 sha256=4a45bebbe355a388d81f955f1440bd237b06336dbbc278d6d6e1197547f76548
  Stored in directory: /root/.cache/pip/wheels/c9/11/90/c7bba8118f3674d75e1457537635266a12538cf622a4684bb2
  Created wheel for marisa-trie: filename=marisa_trie-0.7.5-cp36-cp36m-linux_x86_64.whl size=862271 sha256=d1d0fe34383f4fdf82e4ad7d5d1e60191ac67f27e753e35d219a4fc9d73f27f5
  Stored in directory: /root/.cache/pip/wheels/45/24/79/022624fc914f0e559fe8a1141aaff1f9df810905a13fc75d57
Successfully built wor

In [6]:
%%bash
WORDFILE=glove.840B.300d.zip
if [ -f "$WORDFILE" ]; then
    echo "$WORDFILE exists."
else
    wget http://nlp.stanford.edu/data/glove.840B.300d.zip
fi


glove.840B.300d.zip exists.


In [7]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import zipfile
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from wmd import WMD
import wordfreq
import pickle


In [8]:
import pandas as pd
import zipfile

z = zipfile.ZipFile("./glove.840B.300d.zip")
glove_pd = pd.read_csv(z.open('glove.840B.300d.txt'), sep=" ", quoting=3, header=None, index_col=0)
glove = {key: val.values for key, val in glove_pd.T.items()}
del glove_pd

In [12]:
with open('reaseach_sentiment/combined_data.pickle', 'rb') as handle:
    df = pickle.load(handle)

In [18]:
zip_ref = zipfile.ZipFile("./glove.840B.300d.zip")
zip_ref.extractall("/tmp")
zip_ref.close()

In [21]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

glove_file = datapath('/tmp/glove.840B.300d.txt')
tmp_file = get_tmpfile('W2V.txt')

_ = glove2word2vec(glove_file, tmp_file)

model = KeyedVectors.load_word2vec_format(tmp_file)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [25]:
df['WMD'] = 0
for i in range(len(df)):
    s1 = str(df['text_1'][i]).lower().split()
    s2 = str(df['text_2'][i]).lower().split()
    #print(s1,",",s2)
    dis = model.wmdistance(s1,s2)
    df.at[i,'WMD']=  dis

In [33]:
with open('reaseach_sentiment/combined_data.pickle', 'wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)